# Reference Room Match

In this notebook we will match the rooms from the two distributors.

In [1]:
# Imports
import pandas as pd
import os
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN
# from fuzzywuzzy import fuzz
from rapidfuzz import process, fuzz

In [2]:
# Go one folder back
os.chdir('..')

In [3]:
file_url_reference = "Data/referance_rooms-1737378184366.csv"

df_reference_initial = pd.read_csv(file_url_reference)
df_reference_initial

,hotel_id,lp_id,room_id,room_name
0,13484077,lp23e8ef,1142730702,Double or Twin Room
1,13487663,lp6554de34,1141927122,House
2,13462809,lp6556c3dc,1142722063,Room
3,13530116,lp6555450b,1141968275,Triple Room
4,13530071,lp6557a92c,1142513784,Apartment
...,...,...,...,...
99995,21684,lp6561b025,2168409,Two-Bedroom Suite
99996,21684,lp6561b025,2168411,Deluxe Triple Room
99997,21684,lp6561b025,2168412,Deluxe Queen Room with Two Queen Beds
99998,21684,lp6561b025,2168413,Classic Quadruple Room


In [4]:
file_url_expedia = "Data/updated_core_rooms.csv"

df_expedia_initial = pd.read_csv(file_url_expedia)
df_expedia_initial

,core_room_id,core_hotel_id,lp_id,supplier_room_id,supplier_name,supplier_room_name
0,1,506732,lp7bb6c,200979491,Expedia,Superior Double Room
1,2,509236,lp7c534,200998017,Expedia,"Deluxe Room, Balcony"
2,3,516326,lp7e0e6,201144757,Expedia,Female Dormitory- 3 Beds
3,4,495330,lp78ee2,201028863,Expedia,"Standard Apartment, 2 Bedrooms (6 people)"
4,5,970167,lpecdb7,218116045,Expedia,"Traditional Cottage, 2 Bedrooms, Harbor View"
...,...,...,...,...,...,...
2869051,2912439,193359,lp2f34f,323872346,Expedia,"Deluxe Room, 1 King Bed with Sofa bed"
2869052,2912440,143473,lp23071,230770971,Expedia,Ocean Bay Pool Room
2869053,2912441,1701692958,lp656dc61e,322166812,Expedia,8 Berth Luxury Caravan
2869054,2912442,143473,lp23071,315521742,Expedia,Beach Room


### Data cleaning
Check how many ids are equal in both datasets:

df_reference['hotel_id'] and df_expedia['core_hotel_id']

In [5]:
# Convert the columns to sets
set_1 = set(df_reference_initial['lp_id'])
set_2 = set(df_expedia_initial['lp_id'])

# Find the intersection of the sets
equal_ids = set_1.intersection(set_2)

# Count the number of equal IDs
count_equal_ids = len(equal_ids)

# Display the results
print("Number of equal IDs:", count_equal_ids)

Number of equal IDs: 28638


We will keep only the hotels from the reference that are found in both datasets, because those are the only rooms that will need to be matched.

In [6]:
df_reference = df_reference_initial[df_reference_initial['lp_id'].isin(equal_ids)]
df_reference

,hotel_id,lp_id,room_id,room_name
0,13484077,lp23e8ef,1142730702,Double or Twin Room
1,13487663,lp6554de34,1141927122,House
2,13462809,lp6556c3dc,1142722063,Room
3,13530116,lp6555450b,1141968275,Triple Room
4,13530071,lp6557a92c,1142513784,Apartment
...,...,...,...,...
99988,482128,lp65563022,48212802,Single Room
99989,482128,lp65563022,48212803,Superior Double or Twin Room with Lake View
99990,482128,lp65563022,48212808,Twin Room with Garden View
99991,482128,lp65563022,48212809,Deluxe Double or Twin Room with Lake View


In [7]:
df_expedia = df_expedia_initial[df_expedia_initial['lp_id'].isin(equal_ids)]
df_expedia

,core_room_id,core_hotel_id,lp_id,supplier_room_id,supplier_name,supplier_room_name
1,2,509236,lp7c534,200998017,Expedia,"Deluxe Room, Balcony"
4,5,970167,lpecdb7,218116045,Expedia,"Traditional Cottage, 2 Bedrooms, Harbor View"
6,7,626491,lp98f3b,201681924,Expedia,"Comfort House, 6 Bedrooms, Ocean View"
9,10,627565,lp9936d,201691684,Expedia,"Family House, 4 Bedrooms"
47,48,544769,lp85001,201249503,Expedia,Family Cabin
...,...,...,...,...,...,...
2868987,2912375,1701692924,lp656dc5fc,322141874,Expedia,"Basic Shared Dormitory (10 bunk beds, N2)"
2869025,2912413,840974,lpcd50e,323988812,Expedia,Presidential Suite
2869026,2912414,840974,lpcd50e,323988824,Expedia,Presidential Villa
2869033,2912421,840974,lpcd50e,323988797,Expedia,Superior Twin


### Room name preprocesing
Cleaning, feature extraction, and normalization.

In [8]:
# Load SpaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Use nlp.pipe() to process multiple room names at once
def preprocess_batch(texts):
    """Efficiently process a batch of text using nlp.pipe()."""
    return [" ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])
            for doc in nlp.pipe(texts, batch_size=1000, n_process=8)]  # Adjust batch_size & n_process


In [9]:
# Apply batch processing in df_reference
df_reference = df_reference.copy()
df_reference["processed_room_name"] = preprocess_batch(df_reference["room_name"].astype(str).tolist())

In [10]:
# Apply batch processing in df_expedia
df_expedia = df_expedia.copy()
df_expedia["processed_room_name"] = preprocess_batch(df_expedia["supplier_room_name"].astype(str).tolist())

In [11]:
df_reference

,hotel_id,lp_id,room_id,room_name,processed_room_name
0,13484077,lp23e8ef,1142730702,Double or Twin Room,double Twin Room
1,13487663,lp6554de34,1141927122,House,House
2,13462809,lp6556c3dc,1142722063,Room,room
3,13530116,lp6555450b,1141968275,Triple Room,Triple room
4,13530071,lp6557a92c,1142513784,Apartment,apartment
...,...,...,...,...,...
99988,482128,lp65563022,48212802,Single Room,Single room
99989,482128,lp65563022,48212803,Superior Double or Twin Room with Lake View,Superior Double Twin Room Lake View
99990,482128,lp65563022,48212808,Twin Room with Garden View,Twin Room Garden view
99991,482128,lp65563022,48212809,Deluxe Double or Twin Room with Lake View,Deluxe Double Twin Room Lake View


In [12]:
df_expedia

,core_room_id,core_hotel_id,lp_id,supplier_room_id,supplier_name,supplier_room_name,processed_room_name
1,2,509236,lp7c534,200998017,Expedia,"Deluxe Room, Balcony",Deluxe Room Balcony
4,5,970167,lpecdb7,218116045,Expedia,"Traditional Cottage, 2 Bedrooms, Harbor View",Traditional Cottage 2 bedroom Harbor View
6,7,626491,lp98f3b,201681924,Expedia,"Comfort House, 6 Bedrooms, Ocean View",Comfort House 6 bedroom Ocean View
9,10,627565,lp9936d,201691684,Expedia,"Family House, 4 Bedrooms",Family House 4 bedroom
47,48,544769,lp85001,201249503,Expedia,Family Cabin,Family Cabin
...,...,...,...,...,...,...,...
2868987,2912375,1701692924,lp656dc5fc,322141874,Expedia,"Basic Shared Dormitory (10 bunk beds, N2)",Basic Shared Dormitory 10 bunk bed N2
2869025,2912413,840974,lpcd50e,323988812,Expedia,Presidential Suite,Presidential Suite
2869026,2912414,840974,lpcd50e,323988824,Expedia,Presidential Villa,Presidential Villa
2869033,2912421,840974,lpcd50e,323988797,Expedia,Superior Twin,Superior Twin


In [13]:
df_reference[df_reference['lp_id'] == 'lp9bfad']

,hotel_id,lp_id,room_id,room_name,processed_room_name
43,7855856,lp9bfad,785585601,One-Bedroom Apartment,bedroom apartment


In [14]:
df_expedia[df_expedia['lp_id'] == 'lp9bfad']

,core_room_id,core_hotel_id,lp_id,supplier_room_id,supplier_name,supplier_room_name,processed_room_name
249498,252011,638893,lp9bfad,201720281,Expedia,"Standard Room, 1 Queen Bed, Garden View",Standard Room 1 Queen Bed Garden View
256820,259333,638893,lp9bfad,201720261,Expedia,"Economy Room, 1 Double Bed",Economy Room 1 Double bed
262292,264805,638893,lp9bfad,201720294,Expedia,"Family Suite, 1 Bedroom, Connecting Rooms, Cit...",Family Suite 1 bedroom connect Rooms City View
267591,270104,638893,lp9bfad,201720284,Expedia,"Executive Room, 1 King Bed, Terrace",Executive Room 1 King Bed terrace
271171,273684,638893,lp9bfad,201720275,Expedia,"Standard Room, 1 Queen Bed, Mountain View (Pri...",Standard Room 1 Queen Bed Mountain View privat...
272972,275485,638893,lp9bfad,201720288,Expedia,"Deluxe Suite, 1 King Bed, Terrace",Deluxe Suite 1 King Bed terrace
274284,276797,638893,lp9bfad,201720313,Expedia,"Apartment, 2 Bedrooms, Kitchenette",apartment 2 bedroom Kitchenette
275229,277742,638893,lp9bfad,201720315,Expedia,"Panoramic Apartment, 2 Bedrooms, Kitchen, Lake...",Panoramic Apartment 2 bedroom Kitchen Lake View


To merge room records from different hotel entries, we will use fuzzy string matching to determine the similarity between room names. Similar room names will then be matched to create a single, consistent room list.

In [15]:
def merge_rooms_by_hotel(lp_id, df1, df2, room_col="processed_room_name", similarity_threshold=80):
    """
    Merge room data for a given hotel id (lp_id) from two DataFrames using fuzzy matching.
    Prevents duplication by ensuring unique room descriptions before merging.
    
    Additionally, returns a DataFrame with the unmatched rooms from df2.

    Parameters:
        lp_id (str/int): The hotel id to filter on.
        df1 (DataFrame): First dataset containing hotel and room info.
        df2 (DataFrame): Second dataset containing hotel and room info.
        room_col (str): The column name for room names in both DataFrames.
        similarity_threshold (int): Minimum similarity score (0-100) for a match.

    Returns:
        tuple: A tuple with two DataFrames:
            - Merged DataFrame with fuzzy-matched room info.
            - DataFrame containing unmatched rooms from df2.
    """
    # Filter for the given hotel
    subset1 = df1[df1['lp_id'] == lp_id].copy()
    subset2 = df2[df2['lp_id'] == lp_id].copy()

    # Deduplicate room names before matching
    unique_rooms1 = subset1[[room_col]].drop_duplicates()
    unique_rooms2 = subset2[[room_col]].drop_duplicates()

    # Create a mapping from df2 to df1 using fuzzy matching
    matched_rooms = {}
    for room2 in unique_rooms2[room_col]:
        match, score, _ = process.extractOne(room2, unique_rooms1[room_col].tolist(), scorer=fuzz.token_sort_ratio)
        if score >= similarity_threshold:
            matched_rooms[room2] = match

    # Apply fuzzy match mapping and ensure no missing values
    subset2['matched_room'] = subset2[room_col].map(matched_rooms).fillna(subset2[room_col])

    # Merge unique room records from df1 and df2 based on the matched room names
    merged_rooms = unique_rooms1.merge(
        subset2[[room_col, 'matched_room']].drop_duplicates(), 
        left_on=room_col, 
        right_on="matched_room", 
        suffixes=('_df1', '_df2')
    )

    # Map back to original DataFrames to include all columns
    final_merged = merged_rooms.merge(
        subset1, left_on="matched_room", right_on=room_col, how="left"
    )
    
    # Identify unmatched rooms in df2 (those not present in the matched_rooms mapping)
    unmatched_rooms_df = subset2[~subset2[room_col].isin(matched_rooms.keys())].copy()
    
    return final_merged, unmatched_rooms_df

### Testing different cases

0. Hotel has rooms with quite similar descriptions in both providers

In [16]:
matched_0, unmatched_0 = merge_rooms_by_hotel('lp7c534', df_reference, df_expedia)
matched_0

,processed_room_name_df1,processed_room_name_df2,matched_room,hotel_id,lp_id,room_id,room_name,processed_room_name
0,Deluxe Room Balcony,Deluxe Room Balcony,Deluxe Room Balcony,13518170,lp7c534,1142485350,"Deluxe Room, Balcony",Deluxe Room Balcony
1,Standard Room,Standard Room,Standard Room,13518170,lp7c534,1142486049,Standard Room,Standard Room


In [17]:
unmatched_0

,core_room_id,core_hotel_id,lp_id,supplier_room_id,supplier_name,supplier_room_name,processed_room_name,matched_room


1. Hotel has the same descriptions for many rooms.

In [18]:
matched_1, unmatched_1 = merge_rooms_by_hotel('lp98f3b', df_reference, df_expedia)
matched_1

,processed_room_name_df1,processed_room_name_df2,matched_room,hotel_id,lp_id,room_id,room_name,processed_room_name
0,Comfort House 6 bedroom Ocean View,Comfort House 6 bedroom Ocean View,Comfort House 6 bedroom Ocean View,13929222,lp98f3b,1143605967,"Comfort House, 6 Bedrooms, Ocean View",Comfort House 6 bedroom Ocean View
1,Comfort House 6 bedroom Ocean View,Comfort House 6 bedroom Ocean View,Comfort House 6 bedroom Ocean View,13929222,lp98f3b,1143606069,"Comfort House, 6 Bedrooms, Ocean View",Comfort House 6 bedroom Ocean View
2,Comfort House 6 bedroom Ocean View,Comfort House 6 bedroom Ocean View,Comfort House 6 bedroom Ocean View,13929222,lp98f3b,1143606107,"Comfort House, 6 Bedrooms, Ocean View",Comfort House 6 bedroom Ocean View
3,Comfort House 6 bedroom Ocean View,Comfort House 6 bedroom Ocean View,Comfort House 6 bedroom Ocean View,13929222,lp98f3b,1143606131,"Comfort House, 6 Bedrooms, Ocean View",Comfort House 6 bedroom Ocean View


In [19]:
unmatched_1

,core_room_id,core_hotel_id,lp_id,supplier_room_id,supplier_name,supplier_room_name,processed_room_name,matched_room


2. Hotel has only 1 room with almost same definition in both providers.

In [20]:
matched_2, unmatched_2 = merge_rooms_by_hotel('lp9936d', df_reference, df_expedia)
matched_2

,processed_room_name_df1,processed_room_name_df2,matched_room,hotel_id,lp_id,room_id,room_name,processed_room_name
0,Family House 4 bedroom,Family House 4 bedroom,Family House 4 bedroom,13940261,lp9936d,1143591319,"Family House, 4 Bedrooms",Family House 4 bedroom


In [21]:
unmatched_2

,core_room_id,core_hotel_id,lp_id,supplier_room_id,supplier_name,supplier_room_name,processed_room_name,matched_room


3. Hotel has different number of rooms in both providers and descriptions do not align.

In [22]:
matched_3, unmatched_3 = merge_rooms_by_hotel('lp9ad2d', df_reference, df_expedia)
matched_3

,processed_room_name_df1,processed_room_name_df2,matched_room,hotel_id,lp_id,room_id,room_name,processed_room_name


In [23]:
unmatched_3

,core_room_id,core_hotel_id,lp_id,supplier_room_id,supplier_name,supplier_room_name,processed_room_name,matched_room
1166505,1169018,634157,lp9ad2d,201717108,Expedia,"Studio, 1 King Bed with Sofa bed",Studio 1 King Bed Sofa bed,Studio 1 King Bed Sofa bed
1173516,1176029,634157,lp9ad2d,201717110,Expedia,"Suite, 1 Bedroom",Suite 1 bedroom,Suite 1 bedroom
1178108,1180621,634157,lp9ad2d,201717109,Expedia,"Suite, 1 Bedroom",Suite 1 bedroom,Suite 1 bedroom
2440265,2483653,634157,lp9ad2d,322386459,Expedia,"Suite, 1 Bedroom (Mobility Accessible, Roll-In...",Suite 1 bedroom Mobility Accessible roll Shower,Suite 1 bedroom Mobility Accessible roll Shower
2440926,2484314,634157,lp9ad2d,322386458,Expedia,"Suite, 1 Bedroom (Hearing Accessible)",Suite 1 bedroom Hearing accessible,Suite 1 bedroom Hearing accessible
2442411,2485799,634157,lp9ad2d,322386463,Expedia,"Studio, 1 King Bed with Sofa bed (Mobility Acc...",Studio 1 King Bed Sofa bed Mobility Accessible...,Studio 1 King Bed Sofa bed Mobility Accessible...
2442416,2485804,634157,lp9ad2d,322386460,Expedia,"Studio, 1 King Bed with Sofa bed (Mobility/Hea...",Studio 1 King Bed Sofa bed Mobility hearing Ac...,Studio 1 King Bed Sofa bed Mobility hearing Ac...
2442427,2485815,634157,lp9ad2d,322386456,Expedia,"Suite, 1 Bedroom (Mobility Accessible, Roll-In...",Suite 1 bedroom Mobility Accessible roll Shower,Suite 1 bedroom Mobility Accessible roll Shower
2442447,2485835,634157,lp9ad2d,322386465,Expedia,"Studio, 1 King Bed with Sofa bed (Hearing Acce...",Studio 1 King Bed Sofa bed hear accessible,Studio 1 King Bed Sofa bed hear accessible


4. Hotel has different number of rooms in both providers and descriptions do not align.

In [24]:
matched_4, unmatched_4 = merge_rooms_by_hotel('lp9bfad', df_reference, df_expedia)
matched_4

,processed_room_name_df1,processed_room_name_df2,matched_room,hotel_id,lp_id,room_id,room_name,processed_room_name


In [25]:
unmatched_4

,core_room_id,core_hotel_id,lp_id,supplier_room_id,supplier_name,supplier_room_name,processed_room_name,matched_room
249498,252011,638893,lp9bfad,201720281,Expedia,"Standard Room, 1 Queen Bed, Garden View",Standard Room 1 Queen Bed Garden View,Standard Room 1 Queen Bed Garden View
256820,259333,638893,lp9bfad,201720261,Expedia,"Economy Room, 1 Double Bed",Economy Room 1 Double bed,Economy Room 1 Double bed
262292,264805,638893,lp9bfad,201720294,Expedia,"Family Suite, 1 Bedroom, Connecting Rooms, Cit...",Family Suite 1 bedroom connect Rooms City View,Family Suite 1 bedroom connect Rooms City View
267591,270104,638893,lp9bfad,201720284,Expedia,"Executive Room, 1 King Bed, Terrace",Executive Room 1 King Bed terrace,Executive Room 1 King Bed terrace
271171,273684,638893,lp9bfad,201720275,Expedia,"Standard Room, 1 Queen Bed, Mountain View (Pri...",Standard Room 1 Queen Bed Mountain View privat...,Standard Room 1 Queen Bed Mountain View privat...
272972,275485,638893,lp9bfad,201720288,Expedia,"Deluxe Suite, 1 King Bed, Terrace",Deluxe Suite 1 King Bed terrace,Deluxe Suite 1 King Bed terrace
274284,276797,638893,lp9bfad,201720313,Expedia,"Apartment, 2 Bedrooms, Kitchenette",apartment 2 bedroom Kitchenette,apartment 2 bedroom Kitchenette
275229,277742,638893,lp9bfad,201720315,Expedia,"Panoramic Apartment, 2 Bedrooms, Kitchen, Lake...",Panoramic Apartment 2 bedroom Kitchen Lake View,Panoramic Apartment 2 bedroom Kitchen Lake View


As we can see the similariry function for merging rooms that belong to the same hotel works properly.